In [1]:
import math
import random
import numpy as np
import pandas as pd
import geopandas as gpd
import itertools
import string
import pickle
import treelib
from toydown import GeoUnit, ToyDown
import timeit
from functools import partial
from dask.distributed import Client

In [2]:
def create_tree_from_leaves(leaf_dict):
    """ Given a dictionary, where the keys are the names of leaf nodes (labeled by their path)
        and the corresponding value is the associated attribute counts, this function returns
        the list of GeoUnits that defines the corresponding tree.
    """
    nodes = leaf_dict.copy()
    h = len(list(leaf_dict.keys())[0])
    counts = ["TOTPOP", "VAP"]
    n = len(list(leaf_dict.values())[0][counts[0]])
    level_offsets = [3, 4, 10]
    
    for offset in level_offsets:
        level_names = list(set(list(map(lambda s: s[:-offset], leaf_dict.keys()))))
#         level_counts = [np.zeros(n)]*len(level_names)
        for node in level_names:
            nodes[node] = {c: np.array([v[c] for k, v in leaf_dict.items() if k.startswith(node)]).sum(axis=0) for c in counts}
    return [GeoUnit(k, k[:-3], v) if len(k) == 15 else GeoUnit(k, k[:-1], v) if len(k) == 12 else GeoUnit(k, k[:-6], v) if len(k) == 11 else GeoUnit(k, k[:-3], v) for k, v in nodes.items()]

In [3]:
leaves_dallas_county = np.load("../data/dallas_county_blocks_all_pop_vap.p", allow_pickle=True)
geounits_dc = create_tree_from_leaves(leaves_dallas_county)
geounits_dc.reverse()

In [4]:
geounits_dc[0].attributes["VAP"]

array([1713876,  569832,  657889,  370517,    5740,   90319,     660,
          2075,   16844])

In [14]:
len(leaves_dallas_county)

44113

In [25]:
len(list(filter(lambda x: len(x.name) == 12, geounits_dc)))

1669

In [5]:
# tracts = np.load("../data/dallas_county_tracts_all_vap.p", allow_pickle=True)
counties = np.load("../data/texas_counties_all_pop_vap.p", allow_pickle=True)
del counties["48113"] ## delete extra copy of Dallas County

In [6]:
tx_data = {'TOTPOP': np.array([25145561,  9460921, 11397345,  2886825,    80586,   948426,
                               17920,    33980,   319558]),
           'VAP': np.array([18279737,  6143144,  9074684,  2076282,    61856,   716968,
                            12912,    21205,   172686])}

In [7]:
tx = GeoUnit("48", None, tx_data)

In [8]:
county_geounits = [GeoUnit(geoid, "48", attr) for geoid, attr in counties.items()]

In [9]:
county_geounits.insert(0, tx)

In [10]:
model_all = ToyDown(county_geounits + geounits_dc, 5, 0.5, [1/5, 1/5, 1/5, 1/5, 1/5], gurobi=True)

In [11]:
# model_all.noise_tree(model_all.get_node(model_all.root))

In [12]:
model_all.noise_and_adjust(verbose=False)

Using license file /Users/smaug/gurobi.lic
Academic license - for non-commercial use only


{'48': {'TOTPOP': array([2.51455658e+07, 9.46093306e+06, 1.13973445e+07, 2.88680970e+06,
         8.05901722e+04, 9.48420419e+05, 1.79412155e+04, 3.39551773e+04,
         3.19571490e+05]),
  'VAP': array([1.82797333e+07, 6.14313949e+06, 9.07467326e+06, 2.07629387e+06,
         6.18421981e+04, 7.16983780e+05, 1.29161966e+04, 2.11970975e+04,
         1.72687442e+05])},
 '48023': {'TOTPOP': array([3.74241916e+03, 4.62240496e+02, 3.16015004e+03, 6.05121547e+01,
         3.86721167e+00, 1.60865526e+01, 1.91656758e+01, 1.03660906e+00,
         1.93604217e+01]),
  'VAP': array([2.98700524e+03, 2.77561302e+02, 2.59443760e+03, 6.05121547e+01,
         3.86721167e+00, 1.60865526e+01, 1.41433912e+01, 1.03660906e+00,
         1.93604217e+01])},
 '48001': {'TOTPOP': array([5.84549010e+04, 9.28212050e+03, 3.58016300e+04, 1.22179657e+04,
         2.06302443e+02, 2.84497513e+02, 1.20150247e+01, 2.72508372e+01,
         6.23118916e+02]),
  'VAP': array([4.69697256e+04, 6.71309975e+03, 2.91559649e+04, 1

In [12]:
len(model_all.level())

44366

In [10]:
client = Client(processes=True, threads_per_worker=1, n_workers=2)
client

Client Scheduler: tcp://127.0.0.1:58532 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 2 Cores: 2 Memory: 17.18 GB


In [11]:
height = 5
epsilons = [0.25, 0.5, 1, 2]
splits = [("equal", [1/5, 1/5, 1/5, 1/5, 1/5]), ("top_heavy", [1/2, 1/4, 1/12, 1/12, 1/12]), 
          ("mid_heavy", [1/12, 1/6, 1/2, 1/6, 1/12]), ("bottom_heavy", [1/12, 1/12, 1/12, 1/4, 1/2])]
n_samps = 20

In [13]:
for eps in epsilons:
    for name, eps_split in splits:
        model_all = ToyDown(county_geounits + geounits_dc, height, eps, eps_split)

        def run_model(i):
            return model_all.noise_and_adjust(verbose=False, gurobi=True)

        print("Starting {} runs with eps {} and {} split".format(n_samps, eps, name), flush=True)
        client.scatter(model_all, broadcast=True)
        
        for j in range(int(n_samps/2)):
            adjusteds = client.map(run_model, range(2))
            to_sav = np.array((client.gather(adjusteds)))
            np.save("irving_results/dallas_county_2010_VAP_eps_{}_{}_split_run_{}.npy".format(eps, name, j), to_sav)
            del to_sav
            del adjusteds
            
        del model_all

Starting 20 runs with eps 0.25 and equal split
Starting 20 runs with eps 0.25 and top_heavy split
Starting 20 runs with eps 0.25 and mid_heavy split
Starting 20 runs with eps 0.25 and bottom_heavy split
Starting 20 runs with eps 0.5 and equal split
Starting 20 runs with eps 0.5 and top_heavy split
Starting 20 runs with eps 0.5 and mid_heavy split
Starting 20 runs with eps 0.5 and bottom_heavy split
Starting 20 runs with eps 1 and equal split
Starting 20 runs with eps 1 and top_heavy split
Starting 20 runs with eps 1 and mid_heavy split
Starting 20 runs with eps 1 and bottom_heavy split
Starting 20 runs with eps 2 and equal split
Starting 20 runs with eps 2 and top_heavy split
Starting 20 runs with eps 2 and mid_heavy split
Starting 20 runs with eps 2 and bottom_heavy split


In [13]:
client.restart()

distributed.nanny - WARNING - Restarting worker


Client Scheduler: tcp://127.0.0.1:57884 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 2 Cores: 2 Memory: 17.18 GB


In [16]:
timeit.timeit(partial(model_dc.noise_and_adjust, verbose=False, gurobi=True), number=1)

229.06443670700003